Setup & Imports

In [73]:
!pip install praw requests

 Reddit API Initialization

In [74]:
# OpenRouter API key (replace with yours)
OPENROUTER_API_KEY = "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"

# Reddit API credentials
import praw

reddit = praw.Reddit(
    client_id="5uBZvIgTKH5uE_jEJPIUeQ",
    client_secret="Tr8C5flzhU81yzryqWT7aL8XOtBxpQ",
    user_agent="u/Zealousideal_Wafer84"
)


Fetch Reddit Posts & Comments

In [75]:
def get_reddit_data(username, limit=30):
    try:
        user = reddit.redditor(username)
        posts = [f"[POST] {submission.title}" for submission in user.submissions.new(limit=limit)]
        comments = [f"[COMMENT] {comment.body}" for comment in user.comments.new(limit=limit)]
        return posts + comments
    except Exception as e:
        print("Error fetching Reddit data:", e)
        return []


Prompt Construction

In [76]:
def build_prompt(username, reddit_data):
    # Join Reddit data and truncate to avoid token overflow
    joined_data = "\n".join(reddit_data[:40])[:5000]

    prompt = f"""
You are a UX researcher creating user personas from Reddit activity.

Format the output to follow this structure and style.
Use bullet points and citation quotes to explain each insight.
If a fictional name cannot be inferred, default to the Reddit username: **{username}**

When presenting sliders (personality, motivations), use **visual bars** made of characters (e.g. ▓▓▓░░░ 60%).

---

👤 **Name**:
real name based on comments. If none can be inferred, use: **{username}**
**Age Range**:
Estimate from context as close as possible (e.g. student, professional)
**Occupation**:
Inferred from tech interest, casual tone, or job-related phrases
**Status**:
(Optional – relationship/family indicators)
**Location**:
Infer from mentions of place or cultural context (e.g. Himachal, US, India)
**Tier**:
Early Adopter, Laggard, etc.
**Archetype**:
(e.g. The Skeptical Optimist, The Meme Monk, The Practical Tinkerer)

---

### 🧭 Personality Sliders
Use visual bars (like ▓ or █) and 0–100% estimates with short justifications. Example:

- **Introvert ↔ Extrovert**: ▓▓▓▓░░░░░ (40%)
  *"Posts infrequently and prefers comments to threads."*

- **Intuition ↔ Sensing**: ▓▓▓▓▓▓▓░░ (70%)
  *"Focuses on ideas, metaphors, and broader meaning."*

---

### 🎯 Motivations
Pick 4–6 motivations. For each:
- Name
- Intensity shown with visual bar (e.g. ▓▓▓▓▓▓░░ 75%)
- 1–2 Reddit quotes as evidence

---

### 🌀 Behaviours & Habits
4–6 bullet points, each with a matching quote. Examples:

- Shares screenshots of hikes or photos → *“Evening stroll. [4160x2340]”*
- Fixes SDK paths, updates Android configs → *“Update: changed ANDROID_HOME…”*

---

### 😤 Frustrations
3–5 frustrations. Include emotional tone + a quote for each.

---

### ✅ Goals & Needs
3–4 forward-facing goals inferred from struggles or desires. No fluff — match with quotes if possible.

---

### 💬 Hero Quote
Choose a humorous, self-aware, or emotional quote from the user that sums them up.
Format:
> “quote here”

---

### 📚 Topics Discussed
Bullet points of recurring themes (e.g. Memes, Android SDK, Hiking, Java)

---

### 📌 Summary
Conclude with 2–3 lines that summarize who this person is, their vibe, and what stands out.


---

Reddit content (posts and comments) from user u/{username}:

{joined_data}
"""

    return prompt


Generate Persona

In [77]:
import time

def generate_persona(prompt, retries=2):
    for attempt in range(retries + 1):
        try:
            response = requests.post(
                "https://openrouter.ai/api/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
                    "Content-Type": "application/json",
                    "HTTP-Referer": "https://openrouter.ai",
                    "X-Title": "RedditPersona"
                },
                json={
                    "model": "mistralai/mistral-7b-instruct",
                    "messages": [{"role": "user", "content": prompt}],
                    "temperature": 0.7,
                    "max_tokens": 1000
                },
                timeout=60  # seconds
            )

            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]
            elif response.status_code == 408:
                print("⚠️ Timeout. Retrying...")
                time.sleep(5)
            elif response.status_code == 500:
                print("⚠️ Internal error. Retrying...")
                time.sleep(5)
            else:
                print(f"❌ Error {response.status_code}")
                print(response.text)
                return None

        except requests.exceptions.Timeout:
            print("⚠️ Network timeout — retrying...")
            time.sleep(5)

    print("❌ Failed after retries.")
    return None


Main Execution Pipeline

In [78]:
username = input("Enter Reddit username: ").strip()
reddit_data = get_reddit_data(username)

if reddit_data:
    prompt = build_prompt(username, reddit_data)
    persona = generate_persona(prompt)

    if persona:
        print("\n🎯 Persona Generated:\n")
        print(persona)

        # Optional: save to file
        with open(f"{username}_persona.txt", "w") as f:
            f.write(persona)
            print(f"\n✅ Saved to {username}_persona.txt")
    else:
        print("❌ Persona generation failed.")
else:
    print("❌ No Reddit data found.")


Enter Reddit username: StatisticianFine1187

🎯 Persona Generated:

 👤 **Name**: StatisticianFine1187 (Real name not revealed)
**Age Range**: 25-35 (Assuming based on life experiences and tech interest)
**Occupation**: Data Analyst or Statistician (Inferred from username)
**Status**: Single (No romantic mentions)
**Location**: United States (Based on cultural context)
**Tier**: Early Adopter
**Archetype**: The Bold Explorer

---

### 🧭 Personality Sliders

- **Introvert ↔ Extrovert**: ▓▓▓▓▓▓▓▓ (90%)
  *"StatisticianFine1187 rarely posts content, but engages in comments and discussions."*

- **Intuition ↔ Sensing**: ▓▓▓▓▓▓▓▓▓▓ (95%)
  *"StatisticianFine1187 frequently shares content related to anime, cosplay, and adult content."*

---

### 🎯 Motivations

- **Anime & Cosplay**: ▓▓▓▓▓▓▓▓▓▓ (100%)
  - *"I just copied and pasted info from this site, which seems fairly reputable. Maybe ask the editors?"*
  - *"What most of us would have used the Geass for."*

- **Adult Content**: ▓▓▓▓▓▓▓▓▓▓ (